In [1]:
print('True')

True


In [2]:
# importing os and load_dotenv
import os
from dotenv import load_dotenv

In [3]:
# loading dotenv and getting the openai api key from the .env file
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
# getting the openai key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [5]:
# importing PyPDFLoader from langchain.document_loaders
from langchain.document_loaders import PyPDFLoader

In [6]:
# showing the present working directory
%pwd

'd:\\genai_project\\Interview-Questions-Generator\\research'

In [7]:
%cd ..

d:\genai_project\Interview-Questions-Generator


d:\genai_project\Interview-Questions-Generator\inter\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
%pwd

'd:\\genai_project\\Interview-Questions-Generator'

# Step 1: Ingesting Data

In [9]:
# path of the file daved into a variable
# calling the PyPDFLoader() class and creating an object
# and through the object call a function call functioned as load()
file_path = 'data/SDG.pdf'
loader = PyPDFLoader(file_path)
data = loader.load()

In [10]:
# showing the whole dataset
data
# showing the data through indexing
data[0]
# length of the dataset
len(data)
# printint the dataset through length wise
for i in data:
    print(i.page_content)



IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD 
CAME TOGETHER TO FACE THE FUTURE.
And what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. 
Not just in some faraway place, but in their own cities and towns and villages.
They knew things didn’t have to be this way. They knew we had enough  
food to feed the world, but that it wasn’t getting shared. They knew there 
were medicines for HIV and other diseases, but they cost a lot. They knew  
that earthquakes and floods were inevitable, but that the high death  
tolls were not. 
They also knew that billions of people worldwide shared their hope for a 
better future.
So leaders from these countries created a plan called the Sustainable 
Development Goals (SDGs). This set of 17 goals imagines a future just 15 years 
off that would be rid of poverty and hunger, and safe from the worst effects of 
climate change. It’s an ambitious plan. 
But there’s ample evidence that we can succeed. In the past 15 years, the 
in

In [11]:
# saving the all page content of the whole dataset into a string varible
question_gen = ''

for page in data:
    question_gen += page.page_content

# Step-2 : Chunking 
#### means partioning text into chunking giving chunk_size and chunk_overlap

In [12]:
# importing TokenTextSplitter from langchain.text_splitter. we need it for
# splitting the whole documents because every model has specific token limitations
# or we can say chunk_size
from langchain.text_splitter import TokenTextSplitter

In [13]:
# now we will call the TokenTextSplitter() class and making an object of it
splitter_ques_gen = TokenTextSplitter(
    model_name= "gpt-3.5-turbo",
    chunk_size= 1000,
    chunk_overlap = 200
)

In [14]:
# calling the function split_text() through the object we just created
# and passing the question_gen where all data are saved into single string varible.
chunk_ques_gen = splitter_ques_gen.split_text(question_gen)
# now our whole dataset saved into a list inside the string type data

In [15]:
# importing Document from langchain.docstore.document
from langchain.docstore.document import Document

In [16]:
# now we need to convert the data into documets type
# and this is the procedure to convert the data into a list
document_ques_gen = [Document(page_content = t) for t in chunk_ques_gen]


In [17]:
# creating an obejct of TokenTextSplitter() class
# why we are doing the same thing because if we check the previous splitter,
# the length of the list is 1
# and we use 10000 for chunk size.
# now we will take smaller chunks size and splitting the text
splitter_ans_gen = TokenTextSplitter(
    model_name = 'gpt-3.5-turbo',
    chunk_size = 100,
    chunk_overlap = 50
)

In [18]:
# calling split_documents() function through the object splitter_ans_gen
document_answer_gen = splitter_ans_gen.split_documents(
    document_ques_gen
)

In [19]:
# getting first index of the documents through the page_content attribute
document_answer_gen[0].page_content

'IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough  \nfood to feed the world, but that it wasn’t getting shared'

In [20]:
# length of the document
len(document_answer_gen)

84

In [21]:
# importing ChatOpenAI from from langchain.chat_models
from langchain.chat_models import ChatOpenAI

In [22]:
# remove warnings while running the cell
import warnings
warnings.filterwarnings('ignore')

In [23]:
# creating an object of ChatOpenAI() class
llm_ques_gen_pipeline = ChatOpenAI(
    model = 'gpt-3.5-turbo',
    temperature = 0.3
)

In [24]:
# prompt template designing
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [25]:
# importing PromptTemplate from langchain.prompts
from langchain.prompts import PromptTemplate

In [26]:
# calling the function PromptTemplate() and passing few parameters 
# and saved into a varible called as PROMPT_QUESTIONS
PROMPT_QUESTIONS = PromptTemplate(
    template=prompt_template, 
    input_variables=['text']
    )

In [27]:
# this prompt template is designed for another concept of refinig or making meaningful answers
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)

In [28]:
# calling the function PromptTemplate() and passing few parameters 
# and saved into a varible called as REFINE_PROMPT_QUESTIONS
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [29]:
# importing load_summarize_chain from langchain.chains.summarize
# here load_summarize_chain would be either class or function
# and langchain.chains.summarize is coming from the path langchain/chains/summarize
from langchain.chains.summarize import load_summarize_chain

In [30]:
# calling load_summarize_chain() function and passing few parameters
ques_gen_chain = load_summarize_chain(
    llm = llm_ques_gen_pipeline, 
    chain_type = "refine", 
    verbose = True, 
    question_prompt=PROMPT_QUESTIONS, 
    refine_prompt=REFINE_PROMPT_QUESTIONS
    )

In [33]:
# ques = ques_gen_chain.run(document_ques_gen)

# print(ques)